In [1]:
import torch
import numpy as np

from trainer import load_embeddings
from lib.tokenizer import preprocessor
from lib.config import MODEL_EC, DEVICE
from lib.data_utils import vectorize

Running on: cuda
Reading twitter_2018 - 1grams ...
/home/houyu/.ekphrasis/stats/twitter_2018/counts_1grams.txt
Reading twitter_2018 - 2grams ...
/home/houyu/.ekphrasis/stats/twitter_2018/counts_2grams.txt
Reading twitter_2018 - 1grams ...
/home/houyu/.ekphrasis/stats/twitter_2018/counts_1grams.txt


In [11]:
def sigmoid(x):
    s = 1 / (1 + np.exp(-x))
    return s

In [24]:
labels = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust', 'neutral']
model_path = '/home/houyu/learning/FinalProject/out/model/EmotionClassification_0.5837_2019-05-05_23:38.model'
model_conf = MODEL_EC
max_length = 65  # 85 train 65 dev 58 test
dev_path = '/home/houyu/learning/FinalProject/datasets/new/E-c-En-dev.txt'

# Loading model
model = torch.load(model_path)
model.eval()
word2idx, _, embeddings = load_embeddings(model_conf)

head = 1
cnt = 0
with open(dev_path, 'r') as fin:
    for line in fin.readlines():
        if head == 1:
            head = 0
            continue
        
        items = line[:len(line)-1].split('\t')
        
        input_text = items[1]

        # Pre-processing inputs
        pro_sent = preprocessor(input_text)

        # Embedding and vectorize
        sample = vectorize(pro_sent, word2idx, max_length)

        # Processing to get model inputs
        samples = []
        lengths = []
        samples.append(sample)
        lengths.append(len(pro_sent))

        samples = np.asarray(samples)
        lengths = np.asarray(lengths)

        samples = torch.tensor(samples)
        lengths = torch.tensor(lengths)

        samples = samples.to(DEVICE)
        lengths = lengths.to(DEVICE)


        # Running model
        outputs, attentions = model(samples, lengths)

        posts = outputs.data.cpu().numpy()
        # predicted = np.clip(np.sign(posts), a_min=0, a_max=None)    # 1   1   0   0   0   0   1   0   0   0   1
        # predicted = predicted.astype(np.int32)
        
        pro_sigmoid = sigmoid(posts)
        predicted = np.clip(np.sign(pro_sigmoid - 0.5), a_min=0, a_max=None)
        predicted = predicted.astype(np.int32)
        
        gold_labels = []
        for item in items[2:13]:
            gold_labels.append(int(item))
        
        gold_labels = np.asarray(gold_labels)
        
        acc = 0
        and_ = (predicted & gold_labels).sum()
        or_ = (predicted | gold_labels).sum()
        if(or_ != 0):
            acc = and_/or_
        
        # print(predicted)
        # print(gold_labels)
        # print(acc)
        
        # pro_sigmoid = sigmoid(posts)
        
        # break
        if acc >= 0.9:            # <= 0.5 错误分析
            print(input_text)
            print(pro_sent)
            print(attentions)
            print(posts)
            print(pro_sigmoid)
            print(gold_labels)
            cnt += 1

cnt

Loading word embeddings: ntua_twitter_affect_310 ...
Loaded word embeddings from cache.
@RanaAyyub @rajnathsingh Oh, hidden revenge and anger...I rememberthe time,she rebutted you.
['<user>', '<user>', 'oh', ',', 'hidden', 'revenge', 'and', 'anger', '.', '<repeated>', 'i', 'rememberthe', 'time', ',', 'she', 'rebutted', 'you', '.']
tensor([0.0556, 0.0556, 0.0556, 0.0556, 0.0556, 0.0556, 0.0556, 0.0556, 0.0556,
        0.0556, 0.0556, 0.0556, 0.0556, 0.0556, 0.0556, 0.0556, 0.0556, 0.0556],
       device='cuda:0', grad_fn=<DivBackward1>)
[ 3.89220524 -2.88721609  1.21851552 -3.92386866 -2.88708806 -6.59311152
 -1.94664431 -3.30505133 -2.60656238 -5.10470009 -4.61893034]
[ 0.98000759  0.05278915  0.77180225  0.01938142  0.05279555  0.0013679
  0.12491973  0.0353983   0.06871727  0.00603156  0.00976701]
[1 0 1 0 0 0 0 0 0 0 0]
I'm doing all this to make sure you smiling down on me bro
['i', 'am', 'doing', 'all', 'this', 'to', 'make', 'sure', 'you', 'smiling', 'down', 'on', 'me', 'bro']
ten

@skh4808 @theveteran425FA @TomiLahren Then why'd they wait until now to start getting pissy? 
['<user>', '<user>', '<user>', 'then', 'why', "'", 'd', 'they', 'wait', 'until', 'now', 'to', 'start', 'getting', 'pissy', '?']
tensor([0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625,
        0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625],
       device='cuda:0', grad_fn=<DivBackward1>)
[ 2.86792135 -2.20783639  1.82235014 -4.28541565 -3.1287117  -7.52041531
 -3.85489488 -3.30591035 -1.05521369 -3.90803838 -6.0329566 ]
[  9.46237683e-01   9.90489796e-02   8.60847890e-01   1.35809174e-02
   4.19383384e-02   5.41613961e-04   2.07367130e-02   3.53689827e-02
   2.58225203e-01   1.96845867e-02   2.39265384e-03]
[1 0 1 0 0 0 0 0 0 0 0]
When did it all started? All these depression &amp; anxiety shits?All these suicide thoughts?All of these bad thoughts thts hugging me at night?
['when', 'did', 'it', 'all', 'started', '?', 'all', 'these', 'depression', '&', 'anxiety',

#Spina #Bifida a #neural #tube #birth #defect where the #nervous #system &amp; #spinal #cord do not fully develop #raredisease #June #awareness
['<hashtag>', 'spina', '</hashtag>', '<hashtag>', 'bifida', '</hashtag>', 'a', '<hashtag>', 'neural', '</hashtag>', '<hashtag>', 'tube', '</hashtag>', '<hashtag>', 'birth', '</hashtag>', '<hashtag>', 'defect', '</hashtag>', 'where', 'the', '<hashtag>', 'nervous', '</hashtag>', '<hashtag>', 'system', '</hashtag>', '&', '<hashtag>', 'spinal', '</hashtag>', '<hashtag>', 'cord', '</hashtag>', 'do', 'not', 'fully', 'develop', '<hashtag>', 'rare', 'disease', '</hashtag>', '<hashtag>', 'june', '</hashtag>', '<hashtag>', 'awareness', '</hashtag>']
tensor([0.0208, 0.0208, 0.0208, 0.0208, 0.0208, 0.0208, 0.0208, 0.0208, 0.0208,
        0.0208, 0.0208, 0.0208, 0.0208, 0.0208, 0.0208, 0.0208, 0.0208, 0.0208,
        0.0208, 0.0208, 0.0208, 0.0208, 0.0208, 0.0208, 0.0208, 0.0208, 0.0208,
        0.0208, 0.0208, 0.0208, 0.0208, 0.0208, 0.0208, 0.0208, 0.0208

Threaten to leave your girl shaking in a wet spot ....
['threaten', 'to', 'leave', 'your', 'girl', 'shaking', 'in', 'a', 'wet', 'spot', '.', '<repeated>']
tensor([0.0833, 0.0833, 0.0833, 0.0833, 0.0833, 0.0833, 0.0833, 0.0833, 0.0833,
        0.0833, 0.0833, 0.0833], device='cuda:0', grad_fn=<DivBackward1>)
[ 0.37126055 -1.60947156  0.57196212 -1.64210975 -1.99582136 -5.8888998
 -3.1983192  -3.40217209 -2.13441968 -3.21724963 -5.32445717]
[ 0.59176356  0.16666199  0.63921583  0.16217819  0.11964235  0.00276237
  0.03922902  0.03222765  0.10579615  0.03852173  0.00484738]
[1 0 1 0 0 0 0 0 0 0 0]
@_Buddh_ @rohit_mhpl @indujalali @rajnathsingh We are also wating when terrorism willb history. And one thing kashmir is not India's
['<user>', '<user>', '<user>', '<user>', 'we', 'are', 'also', 'wating', 'when', 'terrorism', 'willb', 'history', '.', 'and', 'one', 'thing', 'kashmir', 'is', 'not', 'india', "'", 's']
tensor([0.0455, 0.0455, 0.0455, 0.0455, 0.0455, 0.0455, 0.0455, 0.0455, 0.0455,
 

@Ginoceno Great prediction bro! Nice optimism #FAKEFAN
['<user>', 'great', 'prediction', 'bro', '!', 'nice', 'optimism', '<hashtag>', 'fakefan', '</hashtag>']
tensor([0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
        0.1000], device='cuda:0', grad_fn=<DivBackward1>)
[-4.49091244 -1.65928066 -4.78178215 -4.5610733   3.17027354 -2.20463753
  3.10395622 -5.71247673 -5.46152782 -3.26869869 -2.17570543]
[ 0.01108613  0.15985858  0.00831139  0.01034275  0.95970023  0.09933481
  0.95705563  0.00329359  0.0042291   0.03666076  0.10195347]
[0 0 0 0 1 0 1 0 0 0 0]
@CNN thanks Hollywood, (Depp). all these climate change fanatics using private jets and gas guzzling cars are to blame for this! #outrage
['<user>', 'thanks', 'hollywood', ',', '(', 'depp', ')', '.', 'all', 'these', 'climate', 'change', 'fanatics', 'using', 'private', 'jets', 'and', 'gas', 'guzzling', 'cars', 'are', 'to', 'blame', 'for', 'this', '!', '<hashtag>', 'outrage', '</hashtag>']
tensor([0.0345, 0.

...

Listening to these refugee children, barely a decade old, describe the horrific acts they were forced to commit is terrifying. #DatelineSBS
['listening', 'to', 'these', 'refugee', 'children', ',', 'barely', 'a', 'decade', 'old', ',', 'describe', 'the', 'horrific', 'acts', 'they', 'were', 'forced', 'to', 'commit', 'is', 'terrifying', '.', '<hashtag>', 'dateline', 'sbs', '</hashtag>']
tensor([0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370,
        0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370,
        0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370],
       device='cuda:0', grad_fn=<DivBackward1>)
[-0.27208719 -2.77346611  1.28510046  2.18261695 -4.2862277  -7.81834793
 -5.37381363 -2.11626339  0.26637456 -2.60914969 -7.48205662]
[  4.32394773e-01   5.87749742e-02   7.83316731e-01   8.98677588e-01
   1.35700433e-02   4.02123958e-04   4.61501908e-03   1.07526131e-01
   5.66202641e-01   6.85518831e-02   5.62781235e-04]

Damn I'm tired as hell I never get a off day during the week anymore 😭 I wanna call in so bad but these lil 60 hrs sounds so good.
['damn', 'i', 'am', 'tired', 'as', 'hell', 'i', 'never', 'get', 'a', 'off', 'day', 'during', 'the', 'week', 'anymore', '😭', 'i', 'wanna', 'call', 'in', 'so', 'bad', 'but', 'these', 'lil', '<number>', 'hrs', 'sounds', 'so', 'good', '.']
tensor([0.0312, 0.0312, 0.0312, 0.0312, 0.0312, 0.0312, 0.0312, 0.0312, 0.0312,
        0.0312, 0.0312, 0.0312, 0.0312, 0.0312, 0.0312, 0.0312, 0.0312, 0.0312,
        0.0312, 0.0312, 0.0312, 0.0312, 0.0312, 0.0312, 0.0312, 0.0312, 0.0312,
        0.0312, 0.0312, 0.0312, 0.0312, 0.0312],
       device='cuda:0', grad_fn=<DivBackward1>)
[ 1.69099152 -3.4707737   1.22368765 -1.25584686 -2.2917099  -5.93208694
 -3.2731142  -0.99517649  0.60228539 -4.17026138 -6.64988136]
[ 0.84435451  0.03015534  0.77271187  0.22168966  0.09181187  0.00264592
  0.03650514  0.26989082  0.64617902  0.01521321  0.0012925 ]
[1 0 1 0 0 0 0 0 1 0 0]
@G

sick of this shit. #mad #angry. Rowan Atkinson Is Not Dead. Just A Bloody Online Hoax😡😡😡😡🤬🤬🤬🤬
['sick', 'of', 'this', 'shit', '.', '<hashtag>', 'mad', '</hashtag>', '<hashtag>', 'angry', '</hashtag>', '.', 'rowan', 'atkinson', 'is', 'not', 'dead', '.', 'just', 'a', 'bloody', 'online', 'hoax', '😡', '😡', '😡', '😡', '\U0001f92c', '\U0001f92c', '\U0001f92c', '\U0001f92c']
tensor([0.0323, 0.0323, 0.0323, 0.0323, 0.0323, 0.0323, 0.0323, 0.0323, 0.0323,
        0.0323, 0.0323, 0.0323, 0.0323, 0.0323, 0.0323, 0.0323, 0.0323, 0.0323,
        0.0323, 0.0323, 0.0323, 0.0323, 0.0323, 0.0323, 0.0323, 0.0323, 0.0323,
        0.0323, 0.0323, 0.0323, 0.0323],
       device='cuda:0', grad_fn=<DivBackward1>)
[ 5.75065708 -5.26702309  4.42676592 -2.65009189 -5.44619703 -9.36393642
 -5.82691622 -3.0881176  -1.59615803 -6.5863018  -9.50043011]
[  9.96829331e-01   5.13246749e-03   9.88188088e-01   6.59833476e-02
   4.29415517e-03   8.57544801e-05   2.93849129e-03   4.36000638e-02
   1.68519273e-01   1.3772314

Shit getting me irritated 😠
['shit', 'getting', 'me', 'irritated', '😠']
tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
       device='cuda:0', grad_fn=<DivBackward1>)
[ 6.81790447 -5.70479822  3.38599896 -4.17851591 -5.7385335  -9.83511257
 -6.31868601 -3.3873291  -1.95634031 -7.49889565 -9.41456985]
[  9.98907208e-01   3.31889745e-03   9.67264175e-01   1.50900315e-02
   3.20915412e-03   5.35354702e-05   1.79906772e-03   3.26938182e-02
   1.23863660e-01   5.53389138e-04   8.15208768e-05]
[1 0 1 0 0 0 0 0 0 0 0]
Never regret anything that once made you smile :) \n #positive 
['never', 'regret', 'anything', 'that', 'once', 'made', 'you', 'smile', '<happy>', '\\', 'n', '<hashtag>', 'positive', '</hashtag>']
tensor([0.0714, 0.0714, 0.0714, 0.0714, 0.0714, 0.0714, 0.0714, 0.0714, 0.0714,
        0.0714, 0.0714, 0.0714, 0.0714, 0.0714],
       device='cuda:0', grad_fn=<DivBackward1>)
[-5.74634266 -2.68499279 -6.25856066 -4.8392148   2.45571947 -0.67355436
  5.10595942 -5.13305998 -4.372027

223

In [23]:
cnt # acc >= 0.9; 0.5 245, 0.3 191, 

191

In [20]:
# 448 <= 0.5

In [25]:
a = {1:[]}

In [26]:
a = set()
a.add(1)
a.add(1)
a

{1}